# CS-6570 Lecture 9 - Variable Selection for Linear Models

You might be tempted to throw all these variables into your model and see what you can predict. However, you'll quickly run into two issues:

- _Model Interpretability_ - If you want to try to understand why and how your model is predicting the way it is, the more input variables you have, the more complicated, convoluted, and contrived your explanation will become.

- _Prediction Accuracy_ - This is even worse. With more variables, your model may start to overfit the data you use to build your model - doing well on data it's seen before, but poorly on the new data that you really want to predict.

Unfortunately, the metrics we use to gauge the fit of a regression model, RSS and its scaled counterpart $R^{2}$, offer no help for this problem. When we add more variables to our model, RSS will _always_ decrease, and the $R^{2}$ value will _always_ increase, at least on the data we're using to build the model (frequently known as the training data).

OK, so if RSS and $R^{2}$ won't allow us to differentiate among potential models - how should we do it? Well, there are a few statistical measures that do take into account the size of our model and penalize larger models. Some of these are:

**Mallow's Cp**

Named after Colin Lingwood Mallows it is defined as:
$\displaystyle C_{p} = \frac{1}{n}\left(RSS + 2d\hat{\sigma}^{2}\right)$

where $\hat{\sigma}^2$ is an estimate of the variance of the error ϵ associated with each response measurement. Typically σ^2 is estimated using the full model containing all predictors.
$\displaystyle \hat{\sigma}^{2} = \frac{RSS}{n-p-1}$

where $p$ is the number of predictors.

**Bayesian Information Criterion (BIC)**

BIC is derived from a Bayesian point of view, and looks similar to Mallow's $C_{p}$. It is defined (up to irrelevant constants) as:

$\displaystyle BIC = \frac{1}{n}\left(RSS + \log{(n)}d\hat{\sigma}^{2}\right)$

Like $C_{p}$, the BIC will tend to take small values for a model with low test error.

**Adjusted $R^{2}$**

Since the $R^{2}$ always increases as more variables are added, the adjusted $R^{2}$ accounts for that fact and introduces a penalty. The intuition is that once all the correct variables have been included in the model, additional noise variables will lead to a very small decrase in RSS, but an increase in k and hence will decrease the adjusted $R^{2}$. In effect, we pay a price for the inclusion of unnecessary variables in the model.
$\displaystyle R^{2}_{a} = 1-\frac{RSS/(n-k-1)}{TSS/n-1}$

$C_{p}$ and $BIC$ have rigorous theoretical justification that rely on asymptotic arguments, i.e. when the sample size $n$ grows very large, they become precise, whereas the adjusted $R^{2}$, although quite intuitive, is not as well motivated in statistical theory. However... it works.

Based on these measures, how can we then determine an optimal subset of predictor variables to use? Well, generally speaking, there are three approaches:

- Check *all* the possible subsets for the best option.
- Find the best possibility with 1 predictor, and then add on the best possible second predictor based on this first predictor, and so on. This is called forward stepwise selection.
- Find the best possibility with all predictors, and then subtract the least importand predictor based on the model with all predictors, and so on. THis is called backward stepwise selection.

**Best Subset Selection**
To perform best subset selection, we fit separate models for each possible combination of the $n$ predictors and then select the best subset. That is we fit:

- All models that contains exactly one predictor
- All models that contain 2 predictors at the second step: $n \choose{2}$
- Until reaching the end point where all $n$ predictors are included in the model

This results in $2^{n}$ possibilities. In our case there are $2^{9} = 512$ possible combinations

_Algorithm_

- Let $M_{0}$ denote the null model which contains no predictors, this model simply predicts the sample mean of each observation

- For $k=1,2,...,n$
    - Fit all $n \choose k$ models that contain exactly k predictors
    - Pick the best among these $n \choose k$ models, and call it $M_{k}$. Here the best is having the smallest $RSS$, or an equivalent measure.
 mo
- Select the single best model among $M_{0},M_{1},...,M_{n}$ using $C_{p}$, $BIC$, adjusted $R^{2}$ or any other method.

**Forward Stepwise Selection**

For computational reasons, the best subset cannot be applied for large $n$ due to the $2^{n}$ complexity. Forward stepwise selection begins with a model containing no predictors, and then adds predictors to the model, one at the time. At each step, the variable that gives the greatest additional improvement to the fit is added to the model.

_Algorithm_

- Let $M_{0}$ denote the null model which contains no predictors.

- For $k=1,2,...,n-1$
    - Consider all $n-k$ that augment the predictors in $M_{k}$ with one additional predictor.
    - Pick the best among these $n - k$ models, and call in $M_{k+1}$.
- Select the single best model among $M_{0},M_{1},...,M_{n}$ using $C_{p}$, $BIC$, adjusted $R^{2}$ or any other method.

In addition to the measures above, there are other widely used methods for adding a "penalty" to the use of many variables. Two popular models along these lines are _ridge regression_ and _lasso_ regression:

For _ridge regression_, the measure we attempt to minimize is:
$\displaystyle \sum_{i = 1}^{n}\left(y_{i} - \hat{y}_{i}\right)^{2} + \lambda \sum_{j = 1}^{p}\beta_{j}^{2}$.

For _lasso regression_, the model we attempt to minimize is:
$\displaystyle \sum_{i = 1}^{n}\left(y_{i} - \hat{y}_{i}\right)^{2} + \lambda \sum_{j = 1}^{p}|\beta_{j}|$.

Both models penalize large values of the coefficients, and so are biased towards $0$. This means the approach will tend to favor underestimating the coefficients. 

Why would we want to do this? Well, if we recall from our first lecture, generally speaking the introduction of bias tends to decrease the variance within our model (the bias-variance tradeoff), and so by decreasing the variance, you decrease the chance that your model will be far removed from the actual "correct" model.

We call the penalty for Ridge regression an $\ell_{2}$ norm, and the penalty for the Lasso is $\ell_{1}$ norm. The different numbers refer to the exponent of the individual $\beta_{i}$ terms. (Note technically to be an $\ell_{2}$ norm, the penalty for ridge regression should have a square root, but don't worry about that here.)

Now, one of the first thing to notice here is that term $\lambda$. What is it, and how do we know its value? Well, the term $\lambda$ is something called a _hyperparameter_. You don't determine it as part of the modeling process - you define it before you start the modeling process. So, you could view both ridge and lasso regression not as single models, but as families of models parametrized by $\lambda$.

At one extreme, $\lambda = 0$, both ridge and lasso regression reduce to standard regression, in which the goal is to minimize $\sum_{i = 1}^{n}\left(y_{i} - \hat{y}_{i}\right)^{2}$. On the other extreme, as $\lambda \rightarrow \infty$, the penalty for any positive coefficient goes to $\infty$, and so the model will tend towards a constant model $Y = \beta_{0}$, where $\beta_{0}$ will just be the average of the output values.

It is between these extremes that we want to build our model, and so how do we figure out $\lambda$? Well, as usual, if we just try to optimize a value like $RSS$ on our training data, we won't get anywhere. Specifically, we'll always choose $\lambda = 0$ by definition.

The validation set approach is conceptually simple and easy to implement. However, it has two potential drawbacks:

1. The validation estimate of the test error rate can be highly variable, and can be highly dependent on precisely which observations are included in the training set vs. the validation set.


2. Only a subset of the observations, those that are included in the training set, are used to fit the model. Since statistical methods tend to perform worse when trained on fewer observations, this suggests that the validation set error rate may _overestimate_ the test error rate for the model when fit on the entire data set.

One way to, partially, address these concerns is by using _cross-validation_, a refinement of the validation set approach that addresses these two issues.

**Leave-One-Out Cross-Valdation (LOOCV)**

For LOOCV, instead of creating two subsets of comparable size as in the validation set approach, a single observation $(x_{i},y_{i})$ is used for the validation set, and the remaining observations make up the training set. The model is then fit on the $n-1$ training observations, and its predicted value $\hat{y}_{i}$ for $x_{i}$ is tested against the actual output $y_{i}$. So, $MSE_{i} = (y_{i}-\hat{y}_{i})^{2}$. This procedure is repeated for all $n$ observations, and the estimated $MSE$ is the average of the errors for each model:

$\displaystyle CV_{(n)} = \frac{1}{n}\sum_{i = 1}^{n}MSE_{i}$.

This approach has far less bias than the validation set approach, and removes the randomness and risk inherent in the training / validation set splits. However, it can be computationally expensive.

**k-Fold Cross-Validation**

An alternative to LOOCV is $k$-fold cross-validation (CV). In this approach, we randomly divide the set of observations into $k$ groups, or _folds_, of approximately equal size. The first fold is treated as the validation set, and the model is fit on the remaining $k-1$ folds. The mean squared error of the model on this first fold, $MSE_{1}$, in then computed. Thes is repeated $k$ times, producing $MSE_{1},MSE_{2},\ldots,MSE_{k}$. The $MSE$ estimate is then computed as:
$\displaystyle CV_{(k)} = \frac{1}{k}\sum_{i = 1}^{n}MSE_{i}$.

It isn't hard to see that the validation set approach and LOOCV are just special cases of $k$-fold CV, with $k = 2$ and $k = n$ respectively. Which value of $k$ should you use for your model? Well, that's another hyperparameter.

The problem with using a large value of $k$ isn't just the increase in computation time, which for modern computers is rarely a huge concern. Yet again, the problem with using a large value for $k$ is the bias-variance trade-off. For the extreme case of LOOCV, the training data for each model is almost exactly the same, which means the models are highly correlated. This means that, while a larger value of $k$ will decrease the bias in the individual estimates (the tendency to overestimate the error), it will also increase the variance. There's basically no way of getting out of that tradeoff.

Alright, now we're going to do a bunch of ridge regressions, and record the coefficients we get. Note, there's no concensus as to whether the hyperparameter for ridge or lasso regression is a $\lambda$ or an $\alpha$. Most of the teaching literature uses $\lambda$, so I've used it here. Unfortunately, Python uses $\alpha$, which you'll see in the code below.

For ridge regression there is already a function, RidgeCV, that will select the best value of $\lambda$ using cross-validation. The value of the parameter "cv" sets the number of folds, and it defaults to LOOCV.

Note that, while many of these coefficients are small, _none_ are $0$. That will not be the case with lasso regression.

Note that for lasso regression here, it figures out the optimal value using iterative fitting, and we need to specify a maximum number of required iterations. We won't get into why it's different than ridge regression, but it has to do with optimizing squared terms vs. optimizing absolute values - smooth derivatives and all that.

What about for larger values of $\lambda$? Well, here we see that lasso sets many coefficients to $0$.

As expected, most of the coefficients are $0$, which is not what we saw for large values of $\lambda$ in ridge regression.